# Analyzing Golang binaries with Radare2

### Load environment

In [4]:
try:
    # if using jupyter within cutter, use the following. This will use the current active binary.
    import cutter
    # we'll assign cutter to variable r2 to be consistent with r2pipe
    r2 = cutter
    print("[+] loaded Cutter binary")
except ModuleNotFoundError as exc:
    # using r2pipe to open a binary
    import r2pipe
    r2 = r2pipe.open("test")
    print("[+] loaded binary")


[+] loaded Cutter binary


### Start analysis

In [5]:
# %time r2.cmd('aaa')

### find .goclntab section

In [7]:
import json

sections = {}

for section in json.loads(r2.cmd("iSj")):
    name = section.get('name')
    sections[name] = { 'name': name, 'size': section.get('size'), 'vaddr': section.get('vaddr')}

In [14]:
size = sections['.gopclntab'].get('size')
vaddr = sections['.gopclntab'].get('vaddr')

# not efficient, takes a long time. Better will be to retrieve directly from 
# raw file.
data = bytes(json.loads(r2.cmd("pxj {} @ {}".format(size, vaddr))))

print ("[+] .gopclntab section found: size={} vaddr={}".format(size, vaddr))

[+] .gopclntab section found: size=448041 vaddr=5024768


In [15]:
import binascii
import struct

def read_var_int(p): 
    result = 0
    shift = 0
    while True:
        b = p[0]
        p = p[1:]

        result |= ((b & 0x7F) << shift)
        if not (b & 0x80):
            break

        shift += 7

    return p, result

def pcValue(p):
    pc = 0
    val = -1

    j = 0
    while True:
        p, uvdelta = read_var_int(p)
        if uvdelta ==0:
            break 

        val += decodeZigZag32(uvdelta)
    
        p, pcdelta = read_var_int(p)
        pc+=(pcdelta)

        yield (val, pc)
        
        j+=1

# https://gist.github.com/matteobertozzi/1521947
def encodeZigZag32(n): return (n << 1) ^ (n >> 31)
def encodeZigZag64(n): return (n << 1) ^ (n >> 63)
def decodeZigZag32(n): return (n >> 1) ^ -(n & 1)
def decodeZigZag64(n): return (n >> 1) ^ -(n & 1)

In [17]:
import binascii
import struct

# https://docs.google.com/document/d/1lyPIbmsYbXnpNj57a261hgOYVpNRcgydurVQIyZOz_o/pub
offset = 0 

magic, = struct.unpack_from("I", s, offset)
offset += 4

if magic != 0xfffffffb:
    print("[!] Magic incorrect for .gopclnt")
    raise Exception("Magic incorrect")
    
quantum, uintptr_size, = struct.unpack_from("xxbb", s, offset)
offset += 4

# https://docs.google.com/document/d/1lyPIbmsYbXnpNj57a261hgOYVpNRcgydurVQIyZOz_o/pub
table_size, = struct.unpack_from("q", s, offset)
offset += 8

# Retrieve path from file table. The file table is located at table_size
def file_table(index):
    file_table_offset, = struct.unpack_from("i", s, 4 + 4 + 8 + table_size * 16 + 8)

    file_table_size, = struct.unpack_from("i", s, file_table_offset)

    name_offset, = struct.unpack_from("i", s, file_table_offset + index * 4)
    name = s[name_offset:]
    name= name[:name.index(b"\0")].decode('utf-8')
    return name

# https://golang.org/src/cmd/internal/objfile/goobj.go
for i in range(0, table_size):
    pc, func_offset, = struct.unpack_from("qq", s, offset)

    offset += struct.calcsize("qq")
            
    entry, name_offset, args, frame, pcsp, pcfile, pcln, nfuncdata, npcdata, = struct.unpack_from("qiiiiiiii", s, func_offset)
    name = s[name_offset:]
    name= name[:name.index(b"\0")].decode('utf-8')
    
    if not name.startswith('main.'):
        continue        

    print("[ ] name={} entry={:x} args={:x} frame={:x} pcsp={:x} pcfile={:x} pcln={:x} nfuncdata={:x} npcdata={:x}".format(name, entry, args, frame, pcsp, pcfile, pcln, nfuncdata, npcdata))

    if pcfile == 0:
        continue

    def line2path(addr):
        for (val, pc) in pcValue(s[pcfile:]):
            if addr < pc:
                return file_table(val)

        return None

    addr = 0
    for (val, pc) in pcValue(s[pcln:]):
        path = line2path(pc)
        if not path:
            continue
            
        r2.cmd('CCu {}:{} @ 0x{:x}'.format(path, val, entry + addr))
        print('[ ] path={}:{} addr=0x{:x}'.format(line2path(pc), val, entry + addr))
        addr = pc

[ ] name=main.main entry=485040 args=0 frame=0 pcsp=6afcb pcfile=6afd4 pcln=6afd8 nfuncdata=3 npcdata=4
[ ] path=/Users/remco/Projects/sans/test.go:5 addr=0x485040
[ ] path=/Users/remco/Projects/sans/test.go:6 addr=0x48505d
[ ] path=/Users/remco/Projects/sans/test.go:7 addr=0x48509d
[ ] name=main.init entry=4850b0 args=0 frame=0 pcsp=6b063 pcfile=6b06e pcln=6b071 nfuncdata=3 npcdata=4


### Print all source files

In [20]:
file_table_offset, = struct.unpack_from("i", s, 4 + 4 + 8 + table_size * 16 + 8)

file_table_size, = struct.unpack_from("i", s, file_table_offset)

for i in range(1, file_table_size):
    name_offset, = struct.unpack_from("i", s, file_table_offset + i * 4)
    name = s[name_offset:]
    name= name[:name.index(b"\0")]
    name = name.decode('utf-8')
    print("[ ] index={} path={}".format(i, name))
    continue

[ ] index=1 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/cpu/cpu.go
[ ] index=2 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/cpu/cpu_x86.go
[ ] index=3 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/cpu/cpu_x86.s
[ ] index=4 path=<autogenerated>
[ ] index=5 path=/usr/local/Cellar/go/1.11.5/libexec/src/sync/atomic/value.go
[ ] index=6 path=/usr/local/Cellar/go/1.11.5/libexec/src/sync/atomic/asm.s
[ ] index=7 path=/usr/local/Cellar/go/1.11.5/libexec/src/runtime/internal/atomic/asm_amd64.s
[ ] index=8 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/bytealg/index_amd64.go
[ ] index=9 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/bytealg/compare_amd64.s
[ ] index=10 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/bytealg/equal_amd64.s
[ ] index=11 path=/usr/local/Cellar/go/1.11.5/libexec/src/internal/bytealg/indexbyte_amd64.s
[ ] index=12 path=/usr/local/Cellar/go/1.11.5/libexec/src/runtime/alg.go
[ ] index=13 path=/usr/local/Cellar/go/1.11.5/lib